# Agent with parse context and structures retrieval, theo cấu trúc định sẵn giữa các sao phụ

https://docs.llamaindex.ai/en/stable/examples/finetuning/react_agent/react_agent_finetune/

https://docs.llamaindex.ai/en/stable/examples/agent/react_agent_with_query_engine/


In [3]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.llms.openai import OpenAI

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [4]:
llm = OpenAI(model="gpt-4o-mini")

In [13]:
def create_and_load_index(files = []):
    persist_dir = "./storage/"
    try:
        storage_context = StorageContext.from_defaults(
            persist_dir=persist_dir,
        )
        index = load_index_from_storage(storage_context)
        index_loaded = True
    except:
        index_loaded = False
    
    if not index_loaded:
        docs = SimpleDirectoryReader(
            input_files=files,
        ).load_data()
        index = VectorStoreIndex.from_documents(docs)
        index.storage_context.persist(persist_dir)

    return index

In [14]:
index_luan = create_and_load_index(["./luan_sao_phu.md"])

In [56]:
from pydantic import BaseModel, Field
import os
from enum import Enum

class CungMenhEnum(str, Enum):
    MENH = "Mệnh"
    PHU = "Phụ"
    PHUC = "Phúc"
    DIEN_TRAI = "Điền Trạch"
    QUAN_LOC = "Quan Lộc"
    NO_BOC = "Nô Bộc"
    THIEN_DI = "Thiên Di"
    TAT_ACH = "Tật Ách"
    TAI_BACH = "Tài Bạch"
    TU_TUC = "Tử Tức"
    PHOI_NGUYEN = "Phối Ngẫu"
    HUYNH_DE = "Huynh Đệ"

class CungMenh(BaseModel):
    """Data model cho cung mệnh"""

    ten_cung: CungMenhEnum = Field(
        description="Tên cung mệnh",
        nullable=True,
    )

    dien_giai: str = Field(
        description="Diễn giải cung mệnh",
        nullable=True,
    )

# Define a Pydantic model for the content
class SaoPhu(BaseModel):
    """Data model cho tử vi : Sao phụ """

    ten_sao_phu: str = Field(
        description="Tên sao phụ",
        nullable=True,
    )

    cung_menh: CungMenhEnum = Field(
        description="Sao này thuộc cung mệnh gì trong 12 cung gì? chỉ cần ghi 1 trong 12 cung",
        nullable=True,
    )

    dien_giai: str = Field(
        description="Tác động của sao phụ này khi nằm ở cung mệnh này",
        nullable=True,
    )

class BocTachSaoPhu(BaseModel):
    """Data model cho bóc tách sao phụ từ nội dung tử vi"""
    sao_phu: list[SaoPhu] = Field(
        description="Phân tích các yếu tố liên quan tới sao phụ này",
    )


# Read the content from the file
file_path = './luan_sao_phu.md'
content = ""
if os.path.exists(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
else:
    print(f"File {file_path} does not exist.")

print(content[:20])

Hóa lộc
Hóa Lộc thuộ


In [57]:

# Split content into smaller chunks
def split_content(content, max_length=1000):
    chunks = []
    while len(content) > max_length:
        split_index = content[:max_length].rfind('\n')
        if split_index == -1:
            split_index = max_length
        chunks.append(content[:split_index])
        content = content[split_index:]
    chunks.append(content)
    return chunks

In [59]:
# cách này không được, dễ quá max length
from llama_index.core.program import FunctionCallingProgram

prompt_template_str = """
# Luan Sao Phu
## Tìm kiếm thông tin về sao phụ
{content}
## Output
{tên sao phụ} : 
    - {cung mệnh 1 [trong 12 cung mệnh] của sao phụ} : {diễn giải tác động số mệnh}
    - {cung mệnh 2 [trong 12 cung mệnh] của sao phụ} : {diễn giải tác động số mệnh}
    - { thêm thông tin cung mệnh nếu có }
"""

# 
programExtract = FunctionCallingProgram.from_defaults(
    output_cls=SaoPhu,
    prompt_template_str=prompt_template_str,
    verbose=True,
)

content_chunks = split_content(content)

# Process each chunk and combine results
combined_results = []
for chunk in content_chunks[:2]:
    response = programExtract(content=chunk)
    combined_results.append(response.dien_giai)

# Combine the results into a single response
final_response = "\n".join(combined_results)
print(final_response)

=== Calling Function ===
Calling function: SaoPhu with args: {"ten_sao_phu": "H\u00f3a L\u1ed9c", "cung_menh": "M\u1ec7nh", "dien_giai": "H\u00f3a L\u1ed9c thu\u1ed9c Th\u1ed5, l\u00e0 th\u1ea7n ph\u00fac \u0111\u1ee9c, h\u1ec9 g\u1eb7p L\u1ed9c T\u1ed3n. Nh\u1eadp mi\u1ebfu \u1edf D\u1ea7n Th\u00e2n, t\u00e1c d\u1ee5ng t\u1ed1t \u0111\u1eb9p r\u1ea5t l\u1edbn, l\u1ea1c h\u00e3m \u1edf cung T\u00fd Ng\u1ecd M\u00e3o D\u1eadu, t\u00e1c d\u1ee5ng t\u1ed1t \u0111\u1eb9p kh\u00e1 nh\u1ecf. Nh\u1eadp m\u1ec7nh, l\u00e0 ng\u01b0\u1eddi hi\u1ec1n h\u00f2a, gi\u1ecfi x\u00e3 giao, nh\u00e2n duy\u00ean t\u1ed1t, t\u00ednh t\u00ecnh h\u00e0o ph\u00f3ng, vui v\u1ebb h\u00e0i h\u01b0\u1edbc, \u1edf ngo\u00e0i \u0111\u01b0\u1ee3c ng\u01b0\u1eddi kh\u00e1c y\u00eau m\u1ebfn, c\u1ea3 \u0111\u1eddi h\u01b0\u1edfng ph\u00fac. H\u1ec9 nh\u1eadp m\u1ec7nh, th\u00e2n, quan, t\u00e0i, di cung, ch\u1ee7 ph\u00fa qu\u00fd, th\u1ee9 \u0111\u1ebfn l\u00e0 nh\u1eadp cung ph\u00fac, \u0111i\u1ec1n."}
=== Functio

In [45]:
query_engine = index_luan.as_query_engine(
    response_mode="tree_summarize", output_cls=BocTachSaoPhu
)

In [47]:
response = query_engine.query("các sao phụ thuộc về các cung mệnh gì thì tác động thế nào ?")
print(str(response))

{"sao_phu":[{"ten_sao_phu":"Mộc Dục","cung_menh":"Tý, Ngọ, Mão, Dậu","dien_giai":"Người tọa mệnh ở các cung Tý, Ngọ, Mão, Dậu có biên độ biến động lớn trong cuộc đời, vận thế thăng trầm dễ tạo thành cục diện hoạch phát hoạch bại. Họ có tính lạc quan, phong lưu, và dễ bộc lộ cảm xúc hỉ, nộ, ai, lạc trên sắc mặt. Cuộc đời của họ đầy nhiều lúc "}]}


In [31]:
from llama_index.core.llms import ChatMessage

sllm = llm.as_structured_llm(output_cls=BocTachSaoPhu)
input_msg = ChatMessage.from_str("sao Thái Dương thuộc về các cung mệnh gì thì tác động thế nào ?")

output = sllm.chat([input_msg])
output_obj = output.raw

In [32]:
print(str(output))

assistant: {"ten_sao_phu":"Thái Dương","cung_menh":"Bạch Dương, Sư Tử, Nhân Mã","dien_giai":"Sao Thái Dương biểu trưng cho ánh sáng, sự sống và năng lượng. Khi thuộc về các cung mệnh như Bạch Dương, Sư Tử, Nhân Mã, nó mang lại sự tự tin, sức mạnh, và khả năng lãnh đạo. Người có sao Thái Dương thường có tính cách mạnh mẽ, sáng tạo và có khả năng thu hút sự chú ý."}
